# MLZoomcamp 2022 - Session #10 - Homework

Author: José Victor

* Dataset:

## Imports

## Building the image

Clone the course repo if you haven't:

ˋˋˋbash
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
ˋˋˋ

Go to the ˋcourse-zoomcamp/cohorts/2022/05-deployment/homeworkˋ folder and execute the following:

ˋˋˋbash
docker build -t zoomcamp-model:v001 .
ˋˋˋ

Note: If you have troubles building the image, you can use the image we built and published to docker hub: ˋsvizor42/zoomcamp-model:v001ˋ.

## Question 1

Run it to test that it's working locally:

ˋˋˋbash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
ˋˋˋ

And in another terminal, execute ˋq6_test.pyˋ file:

ˋˋˋbash
python q6_test.py
ˋˋˋ

You should see this:

ˋˋˋ
{'get_card': True, 'get_card_probability': <value>}
ˋˋˋ

Here ˋ<value>ˋ is the probability of getting a credit card. You need to choose the right one.

* ( ) 0.289
* ( ) 0.502
* ( ) 0.769
* ( ) 0.972

Now you can stop the container running in Docker.

## Installing ˋkubectlˋ and ˋkindˋ

You need to install:

* ˋkubectlˋ - [https://kubernetes.io/docs/tasks/tools/](https://kubernetes.io/docs/tasks/tools/) (you might already have it - check before installing)
* ˋkindˋ - [https://kind.sigs.k8s.io/docs/user/quick-start/](https://kind.sigs.k8s.io/docs/user/quick-start/)

## Question 2

What's the version of ˋkindˋ that you have

Use ˋkind --versionˋ to find out.

### Creating a cluster

Now let's create a cluster with ˋkindˋ:

ˋˋˋ
kind create cluster
ˋˋˋ

And check with ˋkubectlˋ that it was successfully created:

ˋˋˋ
kubectl cluster-info
ˋˋˋ

## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (ˋkindˋ in our case)

* ( ) Node
* ( ) Pod
* ( ) Deployment
* ( ) Service

## Question 4

Now let's test if everything works. Use ˋkubectlˋ to get the list of running services.

What's the ˋTypeˋ of the service that is already running there

* ( ) ClusterIP
* ( ) NodePort
* ( ) LoadBalancer
* ( ) ExternalName

## Question 5

To be able to use the docker image we previously created (ˋzoomcamp-model:v001ˋ), we need to register it with ˋkindˋ.

What's the command we need to run for that

* ( ) ˋkind create clusterˋ
* ( ) ˋkind build node-imageˋ
* ( ) ˋkind load docker-imageˋ
* ( ) ˋkubectl applyˋ

## Question 6

Now let's create a deployment config (e.g. ˋdeployment.yamlˋ):

ˋˋˋyaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64mi"
            cpu: "100m"
          limits:
            memory: <Memory>
            cpu: <CPU>
          ports:
          - containerPort: <Port>
ˋˋˋ

Replace ˋ<Image>ˋ, ˋ<Memory>ˋ, ˋ<CPU>ˋ, ˋ<Port>ˋ with the correct values.

What is the value for ˋ<Port>ˋ

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

## Question 7

Let's create a service for this deployment (ˋservice.yamlˋ)ˋ:

ˋˋˋyaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <>
  ports:
  - port: 80
    targetPort: <PORT>
ˋˋˋ

Fill it in. What do we need to write instead of ˋ<>ˋ

Apply this config file.

### Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

ˋˋˋ
kubectl port-forward service/<Service name> 9696:80
ˋˋˋ

Run ˋq6_test.pyˋ (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.

### Autoscaling

Now we're going to use a [HorizontalPodAutoscaler]() (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scalling the workload to match demand.

Use the following command to create the HPA:

ˋˋˋ
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
ˋˋˋ

You can check the current status of the new HPA by running:

ˋˋˋ
kubectl get hpa
ˋˋˋ

The output should be similar to the next:

ˋˋˋ
NAME            REFERENCE               TARGETS   MINPODS   MAXPODS   REPLICAS  AGE
credit-card-hpa Deployment/credit-card  1%/20%    1         3         1         27s
ˋˋˋ

ˋTARGETˋ column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release from the ˋcomponents.yamlˋ manifest:

ˋˋˋ
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
ˋˋˋ

### Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing ˋq6_test.pyˋ script by putting the operator that sends the request to the credit-card service into a loop.

ˋˋˋpython
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
ˋˋˋ

Now you can run this script.

## Question 8 (optional)

Run ˋkubectl get hpa php-apache --watchˋ command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test

* ( ) 1
* ( ) 2
* ( ) 3
* ( ) 4

Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.